## Scraping Articles

In [1]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Path to chromedriver
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# **executable_path is unpacking the dictionary we’ve stored the path in. 

In [4]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)
# Searching for elements with a specific combination of tag (ul and li) and attribute (item_list and slide).
# Tell browser to wait one second before searching for components. The optional delay is useful because 
#sometimes dynamic pages take a little while to load, especially if they are image-heavy.

True

In [5]:
# Set up the HTML Parser
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

# Assigned slide_elem as the variable to look for the <ul /> tag and its descendent (the other tags within the
# <ul /> element), the <li /> tags. This is the parent element, it holds all of the other elements within it,
# we reference it when we want to filter search results even further. The . is used for selecting classes, such as
# item_list, so the code 'ul.item_list li.slide' pinpoints the <li /> tag with the class of slide and the
# <ul /> tag with a class of item_list. CSS works from right to left, such as returning the last item on the list
# instead of the first. Because of this, when using select_one, the first matching element returned will be
# a <li /> element with a class of slide and all nested elements within it.

In [6]:
# Chain .find onto slide_elem variable to find specific info.
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8744/nasa-engineers-checking-insights-weather-sensors/" target="_self">NASA Engineers Checking InSight's Weather Sensors</a></div>

In [7]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"NASA Engineers Checking InSight's Weather Sensors"

In [8]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

'An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.'

## Scraping Images

In [9]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [10]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [11]:
# Find the more info button by searching for text and click it
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [12]:
# Parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [13]:
# Find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

# tell BeautifulSoup to look inside the <figure class=”lede” /> tag for an <a /> tag, and then look within 
# that <a /> tag for an <img /> tag.

# Pull the link of the image by pointing BeautifulSoup to where the image will be, instead of grabbing the URL
# directly. This way, when NASA updates its image page, the code will still pull the most recent image.

'/spaceimages/images/largesize/PIA18292_hires.jpg'

In [14]:
# Use the base URL to create an absolute URL
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18292_hires.jpg'

## Mars Facts

In [15]:
# Scrape HTML table and store in new df
# The Pandas function read_html() specifically searches for and returns a list of tables found in the HTML. 
# By specifying an index of 0, we’re telling Pandas to pull only the first table it encounters.
df = pd.read_html('http://space-facts.com/mars/')[0]

# Assign columns to the new DataFrame for additional clarity
df.columns=['description', 'value']

# Turn the Description column into the DataFrame’s index. inplace=True means that the updated index will remain 
# in place, without having to reassign the DataFrame to a new variable.
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [16]:
# Convert stored df back into HTML format so it can be added to a website
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [17]:
# End the automated browsing session
browser.quit()

# Challenge

In [19]:
#Assign mars image website to url variable
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

#visit site
browser.visit(url)

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=54826): Max retries exceeded with url: /session/6558b036d967967850a30e21384b186f/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd742755d50>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
hemisphere_images = []
links = browser.find_by_css('a.product-itrm h3')